In [3]:
import google.genai as genai 
from google.genai.types import Tool, GoogleSearch, GenerateContentConfig
from dotenv import load_dotenv
import os
load_dotenv()

True

In [13]:
def create_price_generation_prompt(item, additional_info=None, website=None):
    prompt = f'''
    You are an expert in scraping the internet and providing **real time retail prices**.
    Everytime you get request for any item's price, you use google search tool provided and return the price of the item.

    Please don't use any filler words. Also, no **highlighted headings and texts**.
    Return strictly only
    ```json
    {{
    food_item_1 : price,
    ...
    }}```

    Here is the item:
    {item}
    '''
    if website is not None:
        prompt = prompt + f'''
        Try to search the required rel time information here:
        {website}    
        '''
    if additional_info is not None:
        prompt = prompt + f'''
        Here is some additional things to consider:
        {additional_info}
        '''

    return prompt

In [14]:
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

google_search_tool = Tool(
    google_search=GoogleSearch()
)

In [21]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=create_price_generation_prompt('Onion', 
                                            additional_info='The information should be based on Mumbai.',
                                            website='blinkit.com'),
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=['TEXT']
    )
)

In [22]:
for each in response.candidates[0].content.parts:
    print(each.text)

```json
{
Onion: "₹40-₹46 per kg"
}
```


In [ ]:
def generate_price(item, addtional_info=None, website=None):
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    google_search_tool = Tool(
        google_search=GoogleSearch()
    )
    response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=create_price_generation_prompt(item, 
                                            additional_info=addtional_info,
                                            website=website),
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=['TEXT']
    )
    )
    # print(response)
    return response.candidates[0].content.parts[0].text

In [32]:
response = generate_price(
        item=['Chicken breast (250g)', 'Onion (100g)', 'Eggs (6)', 'Green Chillies (50g)'],
        addtional_info='The information should be based on Mumbai.',
        website=['liscious.com', 'blinkit.com']
    )

print(response)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='```json\n{\n"Chicken breast (250g)": "₹159",\n"Onion (100g)": "₹4.2",\n"Eggs (6)": "₹47 - ₹55",\n"Green Chillies (50g)": "₹8 - ₹9.5"\n}\n```')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=GroundingMetadata(grounding_chunks=None, grounding_supports=None, retrieval_metadata=RetrievalMetadata(google_search_dynamic_retrieval_score=None), retrieval_queries=None, search_entry_point=SearchEntryPoint(rendered_content='<style>\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: so